# SABR Certification Course 3: Final Project

 I'm going to challenge you to calculate a simple set of MARCEL projections and then figure out how to beat MARCEL, using either a different, new set of data or a better method. Let's do it.

**The final assignment**

Now you've seen the basics of how MARCEL works, your final assignment will be to calculate a MARCEL projection, and then beat it. Here's a guide to the rules, a formula, and how to go about it. I encourage you to start with the code I've provided in these tutorials and then modify it as necessary to finish the assignment.

https://docs.google.com/document/d/1iZhoSN2Y5d9xfgx5qYI8gwUfrW1vaXbf4qssWaNXzUA/edit?usp=sharing

The data for the assignment is here:
https://huggingface.co/spaces/rkarthur/sabr3evaluation/blob/main/data/SABR3_data_for_assignment.csv

You should submit your final assignment in the learning management system, providing the email that you'd like to receive feedback at.

There needs to be a column labeled `MARCEL_OPS` that contains your predicted MARCEL OPS, and then a second column labeled `BETTER_THAN_MARCEL_OPS` that contains your improved version of MARCEL's numbers. The application will automatically grade your assignment and let you know if your numbers are matching MARCEL and then, better than MARCEL.

Once you beat MARCEL, imagine you're giving a brief to a data scientist colleague and write up a brief paragraph or so about how you did it. Consider including how you got the data, how you linked it with the existing data, and how much it improved the resulting projections. You'll submit that brief as well for feedback from the Baseball Prospectus graders. Good luck!


In [ ]:
library(tidyverse)
data = read_csv("https://huggingface.co/spaces/rkarthur/sabr3evaluation/resolve/main/data/SABR3_data_for_assignment.csv")
head(data)


In [ ]:
 data1 <- data %>%
  mutate(total_pas = PA1 + PA2 + PA3,
        age_adj = ifelse(Age > 29, (Age - 29)* -0.003, (29 - Age)* 0.006),
        reliability = total_pas/(total_pas + 1200),
        final_age_adj = 1 + age_adj,
        w_OPS = (5*(OPSY1) +  4*(OPSY2) + 3*(OPSY3))/12,
        regr_rate = (reliability * w_OPS) + ((1 - reliability)*.720),
        MARCEL_OPS = final_age_adj * regr_rate)


In [ ]:
data1

In [ ]:
 data2 <- data %>%
  mutate(player_age_adj = Age + 1,
        total_pas = PA1 + PA2 + PA3,
        age_adj = ifelse(player_age_adj > 29, (player_age_adj - 29)* -0.003, (29 - player_age_adj)* 0.006),
        reliability = total_pas/(total_pas + 1200),
        final_age_adj = 1 + age_adj,
        weighted_OPS = (5*(OPSY1) +  4*(OPSY2) + 3*(OPSY3))/12,
        regr_rate = (reliability * weighted_OPS) + ((1 - reliability)*.720),
        MARCEL_OPS = final_age_adj * regr_rate)

In [ ]:
data2

In [ ]:
statcast = read_csv("stats.csv")
head(statcast)

In [ ]:
str(statcast)

In [ ]:
statcast_expanded <- statcast %>%
  group_by(player_id) %>%
  mutate( Z_O = z_swing_percent - oz_swing_percent,
          K_BB = k_percent - bb_percent,
          pa1 = lag(pa, 1),
          pa2 = lag(pa, 2),
          pa3 = lag(pa, 3),
          total_pa = pa1 + pa2 + pa3,
          total_pa2 = pa + pa1 + pa2 + pa3,
          HH1 = lag(hard_hit_percent, 1),
          HH2 = lag(hard_hit_percent, 2),
          HH3 = lag(hard_hit_percent, 3),
          SS1 = lag(sweet_spot_percent, 1),
          SS2 = lag(sweet_spot_percent, 2),
          SS3 = lag(sweet_spot_percent, 3),
          Chase1 = lag(oz_swing_percent, 1),
          Chase2 = lag(oz_swing_percent, 2),
          Chase3 = lag(oz_swing_percent, 3),
          Whiff1 = lag(whiff_percent, 1),
          Whiff2 = lag(whiff_percent, 2),
          Whiff3 = lag(whiff_percent, 3),
          ZO1 = lag(Z_O, 1),
          ZO2 = lag(Z_O, 2),
          ZO3 = lag(Z_O, 3),
          BB1 = lag(bb_percent, 1),
          BB2 = lag(bb_percent, 2),
          BB3 = lag(bb_percent, 3),
          KBB1 = lag(K_BB, 1),
          KBB2 = lag(K_BB, 2),
          KBB3 = lag(K_BB, 3),
          xwoba1 = lag(xwoba, 1),
          xwoba2 = lag(xwoba, 2),
          xwoba3 = lag(xwoba, 3),
          woba1 = lag(woba, 1),
          woba2 = lag(woba, 2),
          woba3 = lag(woba, 3),
          xiso1 = lag(xiso, 1),
          xiso2 = lag(xiso, 2),
          xiso3 = lag(xiso, 3),
          xslg1 = lag(xslg, 1),
          xslg2 = lag(xslg, 2),
          xslg3 = lag(xslg, 3),
          xobp1 = lag(xobp, 1),
          xobp2 = lag(xobp, 2),
          xobp3 = lag(xobp, 3),
          xops = xobp + xslg,
          xops1 = xobp1 + xslg1,
          xops2 = xobp2 + xslg2,
          xops3 = xobp3 + xslg3,
          xwobacon1 = lag(xwobacon, 1),
          xwobacon2 = lag(xwobacon, 2),
          xwobacon3 = lag(xwobacon, 3),
          barrel1 = lag(barrel_batted_rate, 1),
          barrel2 = lag(barrel_batted_rate, 2),
          barrel3 = lag(barrel_batted_rate, 3),
          ev50_1 = lag(avg_best_speed, 1),
          ev50_2 = lag(avg_best_speed, 2),
          ev50_3 = lag(avg_best_speed, 3),
          adj_ev1 = lag(avg_hyper_speed, 1),
          adj_ev2 = lag(avg_hyper_speed, 2),
          adj_ev3 = lag(avg_hyper_speed, 3),
          ops1 = lag(on_base_plus_slg, 1),
          ops2 = lag(on_base_plus_slg, 2),
          ops3 = lag(on_base_plus_slg, 3),
          xwoba_xiso1 = xwoba1 + xiso1,
          xwoba_xiso2 = xwoba2 + xiso2,
          xwoba_xiso3 = xwoba3 + xiso3,
          xwoba_adj1 = (0.74522/0.32135)*xwoba1,
          xwoba_adj2 = (0.74518/0.32046)*xwoba2,
          xwoba_adj3 = (0.74547/0.31856)*xwoba3
          )%>%
            filter(pa1 >= 100 & pa2 >= 100 & pa3 >= 100)%>%
            na.omit()

In [ ]:
statcast_expanded

In [ ]:
if (!requireNamespace('pacman', quietly = TRUE)){
  install.packages('pacman')
}
pacman::p_load_current_gh("BillPetti/baseballr")

install.packages("Lahman")
install.packages("tidyverse")
install.packages("RSQLite")
library(Lahman)
library(baseballr)
library(tidyverse)
library(RSQLite)

In [ ]:
player_key_lookups <- chadwick_player_lu()%>%
  select(key_mlbam, key_bbref, key_bbref_minors, name_last, name_first)%>%
  filter(!is.na(key_mlbam))


player_key_lookups


In [ ]:
final_statcast_df <-statcast_expanded%>%
  left_join(player_key_lookups,
join_by(player_id == key_mlbam))%>%
  select('last_name, first_name', player_id, key_bbref, year, player_age, pa, pa1, pa2, pa3, total_pa, total_pa2,on_base_plus_slg, xops, xops1, xops2, xops3, ops1, ops2, ops3)

In [ ]:
final_statcast_df %>%
  select(player_id, year, player_age, pa, pa1, pa2, pa3, total_pa, total_pa2, on_base_plus_slg, xops, xops1, xops2, xops3)%>%
  cor()

In [ ]:
final_statcast_df

In [ ]:
summary(lm(on_base_plus_slg ~ player_age + (xops1) + (xops2) + (xops3) + total_pa, data = final_statcast_df))

In [ ]:
train1 <- final_statcast_df %>%
  filter(year !=2021)

test_int1 <- final_statcast_df%>%
  filter(year == 2021)

test1 <- data2%>%
  left_join(test_int1,
join_by(bbref_id == key_bbref))%>%
  select('last_name, first_name', bbref_id, player_id, year, player_age, on_base_plus_slg, PA1, PA2, PA3, PA4, OPSY1, OPSY2, OPSY3, OPSY4, total_pa,
reliability, final_age_adj, weighted_OPS, regr_rate, MARCEL_OPS,xops, xops1, xops2, xops3)%>%
  na.omit()


model1 <- lm(on_base_plus_slg ~ player_age + (xops1) + (xops2) + (xops3) + total_pa, data = train1)

test1$BETTER_THAN_MARCEL_OPS <- predict(model1, test1)


#summary(model)

cor(test2$BETTER_THAN_MARCEL_OPS, test2$on_base_plus_slg)


[1] 0.5883613

In [ ]:
summary(model1)

In [ ]:
write_csv(test1, "marcel_data_submission.csv")

In [ ]:
sqrt(mean((test1$MARCEL_OPS - test1$on_base_plus_slg)**2))

[1] 0.1114393

In [ ]:
sqrt(mean((test1$BETTER_THAN_MARCEL_OPS - test1$on_base_plus_slg)**2))

[1] 0.1050524

In [ ]:
median(abs(test1$MARCEL_OPS - test1$on_base_plus_slg))

[1] 0.06298856

In [ ]:
median(abs(test1$BETTER_THAN_MARCEL_OPS - test1$on_base_plus_slg))

[1] 0.06576642